# get pdf

In [1]:
from get_pdf import get_jaist_lecture_pdf

In [2]:
YEAR = 2021

In [3]:
file_names = get_jaist_lecture_pdf(year=YEAR)

In [4]:
file_names

['pdfs/calendar2021_4-8-1.pdf',
 'pdfs/calendar2021_9-12_4.pdf',
 'pdfs/calendar2021_1-3.pdf']

In [5]:
ls pdfs

calendar2021_1-3.pdf     calendar2021_4-8-1.pdf   calendar2021_9-12_4.pdf


# convert pdf to xlsx

このサイト使って変換する

[sodapdf](https://www.sodapdf.com/pdf-converter/)

In [6]:
ls xlsxs/

calendar2021_1-3.xlsx       calendar2021_9-12_4.xlsx
calendar2021_4-8-1.xlsx     ~$calendar2021_9-12_4.xlsx


In [7]:
xlsx_fnames = [f.replace('pdf', 'xlsx') for f in file_names]
xlsx_fnames

['xlsxs/calendar2021_4-8-1.xlsx',
 'xlsxs/calendar2021_9-12_4.xlsx',
 'xlsxs/calendar2021_1-3.xlsx']

# convert pdf to pandas_dataframe

In [8]:
from read_xlsx import preprocess_xlsx, get_unique_lecture

In [9]:
df = preprocess_xlsx(xlsx_fnames)

In [15]:
df.reset_index().groupby(['year','month'])['day'].nunique().reset_index()

,year,month,day
0,2021,4,30
1,2021,5,31
2,2021,6,30
3,2021,7,31
4,2021,8,31
5,2021,9,30
6,2021,10,31
7,2021,11,30
8,2021,12,31
9,2022,1,31


## get unique lecture

In [16]:
leclist = get_unique_lecture(df)

In [237]:
#leclist

## get lecture day and time

In [21]:
import pandas as pd
from datetime import datetime

In [37]:
lecdict = {}
for lec in leclist:
    lectime = []
    for index, row in df[df==lec].dropna(how='all').iterrows():
        for col in row.dropna().index:
            lectime.append([index,col])
    lecdict.update({lec: lectime})

In [38]:
# lecdict

In [104]:
df_lec = pd.DataFrame.from_dict(lecdict, orient='index').stack().reset_index(0)
df_lec.columns = ['title', 0]

srs_date =  df_lec[0].str[0].str
df_lec['Start Date'] = df_lec['End Date'] = df_lec[0].str[0].apply(lambda datetuple: datetime(*datetuple)).dt.strftime('%Y/%m/%d')

In [106]:
d={int(i[0]): i.split('\n')[1].split(' - ') for i in [
    '1限\n09:20 - 11:00', '2限\n11:10 - 12:50', '3限\n13:50 - 15:30',
    '4限\n15:40 - 17:20', '5限\n17:30 - 19:10', '6限\n18:30 - 20:10',
    '7限\n20:15 - 21:55']}

d

{1: ['09:20', '11:00'],
 2: ['11:10', '12:50'],
 3: ['13:50', '15:30'],
 4: ['15:40', '17:20'],
 5: ['17:30', '19:10'],
 6: ['18:30', '20:10'],
 7: ['20:15', '21:55']}

In [107]:
df_lec['Time'] = df_lec[0].str[1].map(d)
df_lec['Start Time'] = df_lec['Time'].str[0]+':00'
df_lec['End Time'] = df_lec['Time'].str[1]+':00'

df_lec['Subject'] = df_lec['title'].str.split('」').str[0].str.replace('\n','').str.replace('「','-')
df_lec['Description'] = '講師: ' + df_lec['title'].str.split('」').str[1]

df_lec = df_lec[['Subject',  'Start Date', 'End Date',  'Start Time', 'End Time', 'Description']]

df_lec = df_lec.drop_duplicates().reset_index(drop=True)

df_lec = df_lec.groupby(['Subject',  'Start Date', 'End Date','Description']).agg({'Start Time':min, 'End Time':max}).reset_index()

df_lec = df_lec[['Subject', 'Start Date', 'End Date', 'Start Time', 'End Time', 'Description']]

check diff

In [53]:
import difflib

In [50]:
import numpy as np

In [110]:
df_lec_prev = pd.read_csv('../jaist-lecture.csv', dtype=str).fillna('')

In [111]:
df_lec = df_lec.sort_values(['Subject', 'Start Date']).reset_index(drop=True)a
df_lec_prev = df_lec_prev.sort_values(['Subject', 'Start Date']).reset_index(drop=True)

In [112]:
d = difflib.Differ()

diff = d.compare([' '.join(l) for l in df_lec.values],
                 [' '.join(l) for l in df_lec_prev.values])

In [113]:
print('\n'.join(diff))

- I211-数理論理学 2021/10/30 2021/10/30 09:20:00 12:50:00 講師: 小川
?                                                      ^^

+ I211-数理論理学 2021/10/30 2021/10/30 09:20:00 12:50:00 講師：小川・横山
?                                                      ^  +++

- I211-数理論理学 2021/11/06 2021/11/06 09:20:00 12:50:00 講師: 小川
- I211-数理論理学 2021/11/13 2021/11/13 09:20:00 12:50:00 講師: 小川
?                                                      ^^

+ I211-数理論理学 2021/11/13 2021/11/13 09:20:00 12:50:00 講師：小川・横山
?                                                      ^  +++

- I211-数理論理学 2021/11/20 2021/11/20 09:20:00 12:50:00 講師: 小川
?                                                      ^^

+ I211-数理論理学 2021/11/20 2021/11/20 09:20:00 12:50:00 講師：小川・横山
?                                                      ^  +++

- I211-数理論理学 2021/11/27 2021/11/27 09:20:00 12:50:00 講師: 小川
- I211-数理論理学 2021/12/04 2021/12/04 09:20:00 12:50:00 講師: 小川
?                                                      ^^

+ I211-数理論理学 2021/12/04 2021/

In [115]:
df_lec = df_lec.sort_values(['Start Date', 'Start Time', "Subject"]).reset_index(drop=True)

In [117]:
df_lec.to_csv('../jaist-lecture.csv', index=False)